<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.5 - Prompting Large Language Models

In this lab we will practise prompting with a few Large Language Models (LLMs) using Groq (not to be confused with Grok). Groq is a platform that provides access to their custom-built AI hardware via APIs, allowing users to run open-source models such as Llama.

We shall see that while LLMs are powerful tools, how you ask a question or frame a task can dramatically influence the results obtained.

## Set-up

Step 1: Sign up for a free Groq account at https://console.groq.com/home .

Step 2: Create a new API key at https://console.groq.com/keys. Copy-paste it into an empty text file called 'groq_key.txt'.

Running the next cell will then read in this key and assign it to the variable `groq_key`.

In [2]:
from google.colab import files

# This will prompt you to upload a file
uploaded = files.upload()


Saving groq_key.txt to groq_key.txt


In [3]:
import os
print(os.getcwd())

/content


In [4]:
groqfilename = 'groq_key.txt' # this file contains a single line containing your Groq API key only
try:
    with open(groqfilename, 'r') as f:
        groq_key = f.read().strip()
except FileNotFoundError:
    print("'%s' file not found" % groqfilename)

In [5]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 687.3 kB/s eta 0:00:00


In [6]:
from groq import Groq
import requests
import pandas as pd
from IPython.display import Markdown

First create an instance of the Groq client:

In [7]:
client = Groq(api_key=groq_key)

The following code shows what models are currently accessible through Groq. `context_window` refers to the size of memory (in tokens) during a session and `max_completion_tokens` is the maximum number of tokens that are generated in an output.

In [8]:
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {groq_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

pd.DataFrame(response.json()['data']).sort_values(['created'], ascending=False)

,id,object,created,owned_by,active,context_window,public_apps,max_completion_tokens
12,moonshotai/kimi-k2-instruct,model,1752435491,Moonshot AI,True,131072,None,16384
7,meta-llama/llama-prompt-guard-2-86m,model,1748632165,Meta,True,512,None,512
15,meta-llama/llama-prompt-guard-2-22m,model,1748632101,Meta,True,512,None,512
3,qwen/qwen3-32b,model,1748396646,Alibaba Cloud,True,131072,None,40960
17,meta-llama/llama-guard-4-12b,model,1746743847,Meta,True,131072,None,1024
0,meta-llama/llama-4-maverick-17b-128e-instruct,model,1743877158,Meta,True,131072,None,8192
6,meta-llama/llama-4-scout-17b-16e-instruct,model,1743874824,Meta,True,131072,None,8192
10,compound-beta-mini,model,1742953279,Groq,True,131072,None,8192
9,compound-beta,model,1740880017,Groq,True,131072,None,8192
20,playai-tts-arabic,model,1740682783,PlayAI,True,8192,None,8192


The Groq client object enables interaction with the Groq REST API and a chat completion request is made via the client.chat.completions.create method.

The most important arguments of the client.chat.completions.create method are the following:
* messages: a list of messages (dictionary form) that make up the conversation to date
* model: a string indicating which model to use (see [list of models](https://console.groq.com/docs/models))
* max_completion_tokens: the maximum number of tokens that are generated in the chat completion
* response_format: setting this to `{ "type": "json_object" }` enables JSON output
* seed: sample deterministically as best as possible, though identical outputs each time are not guaranteed
* temperature: between 0 and 2 where higher values like 0.8 make the output more random (creative) and values like 0.2 are more focused and deterministic


In [9]:
# help(client.chat.completions.create)

As a first example, note how the messages input is given as a list of a dictionaries with `role` and `content` keys. This is in a ChatML format recognised by many LLMs.

In [10]:
chat_completion = client.chat.completions.create(
    messages=[
        {   "role": "system", # sets the persona of the model
            "content": "You are a helpful assistant."
        },
        {
            "role": "user", # what the user wants the assistant to do
            "content": "Explain briefly how large language models work",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training on vast amounts of text data**: Models are trained on massive datasets of written language, such as books, articles, and conversations.
2. **Learning patterns and relationships**: Through complex algorithms, the models identify patterns, relationships, and context within the training data.
3. **Generating text based on probability**: When given a prompt or input, the model predicts the next word or character based on the patterns and relationships it learned during training.
4. **Iterating and refining**: The model continues to generate text, refining its output based on the input and its internal understanding of language.

This process allows large language models to generate coherent and often contextual text, simulating human-like conversation and writing.


The output is in Markdown format so the following line formats this text.

In [11]:
Markdown(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training on vast amounts of text data**: Models are trained on massive datasets of written language, such as books, articles, and conversations.
2. **Learning patterns and relationships**: Through complex algorithms, the models identify patterns, relationships, and context within the training data.
3. **Generating text based on probability**: When given a prompt or input, the model predicts the next word or character based on the patterns and relationships it learned during training.
4. **Iterating and refining**: The model continues to generate text, refining its output based on the input and its internal understanding of language.

This process allows large language models to generate coherent and often contextual text, simulating human-like conversation and writing.

## Text summarisation

We start with a llama3-8b-8192, a model using just over 8 billion parameters with at most 8192 tokens produced as output.

Here is an article to be summarised from the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset:

In [12]:
story = """
SAN FRANCISCO, California (CNN) -- A magnitude 4.2 earthquake shook the San Francisco area Friday at 4:42 a.m. PT (7:42 a.m. ET), the U.S. Geological Survey reported. The quake left about 2,000 customers without power, said David Eisenhower, a spokesman for Pacific Gas and Light. Under the USGS classification, a magnitude 4.2 earthquake is considered "light," which it says usually causes minimal damage. "We had quite a spike in calls, mostly calls of inquiry, none of any injury, none of any damage that was reported," said Capt. Al Casciato of the San Francisco police. "It was fairly mild." Watch police describe concerned calls immediately after the quake » . The quake was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles, the USGS said. Oakland is just east of San Francisco, across San Francisco Bay. An Oakland police dispatcher told CNN the quake set off alarms at people's homes. The shaking lasted about 50 seconds, said CNN meteorologist Chad Myers. According to the USGS, magnitude 4.2 quakes are felt indoors and may break dishes and windows and overturn unstable objects. Pendulum clocks may stop.
"""

**Exercise:**
Summarise the story text using the following three prompts. Use the format given above but here there is no need to set the persona (i.e. only include one dictionary in the messages list when calling `client.chat.completions.create`.) Comment on any differences.

1) "Summarise the following article in 3 sentences."

2) "Give me a TL;DR of this text."

3) "What's the key takeaway here?"

In [13]:
prompts = ["Summarise the following article in 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"]
#content will be p + story for p in prompts

# ANSWER
for p in prompts:
    response = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[{"role": "user", "content": p + story}]
)

    print(p, '\n', response.choices[0].message.content)

Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area on Friday morning, with its epicenter located 2 miles east-northeast of Oakland at a depth of 3.6 miles. The quake caused minimal damage and disruption, with about 2,000 customers losing power and no reports of injury or damage. The shaking lasted around 50 seconds and was felt indoors, with some people experiencing alarm going off at their homes and pendulum clocks stopping due to the quake.
Give me a TL;DR of this text.  
 Here is a brief summary of the text:

A magnitude 4.2 earthquake occurred in the San Francisco area on Friday morning, causing minimal damage and no reported injuries. The quake left around 2,000 customers without power, but authorities say it was "fairly mild" and mostly caused concern among residents, rather than damage.
What's the key takeaway here? 
 The key takeaway is that a magnitude 4.2 earthquake stru

Run the above code again below and note that the answers may differ. This is due to the probabilistic nature of LLM token generation.

In [15]:
# ANSWER
for p in prompts:
    response = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[{"role": "user", "content": p + story}]
)

    print(p, '\n', response.choices[0].message.content)

Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area on Friday morning, causing minimal damage and leaving around 2,000 customers without power. The quake was considered "light" by the USGS, with no reports of injuries or significant damage, according to police. The shaking lasted around 50 seconds and was felt indoors, causing some objects to fall and overturn, but some people's homes were also set off by alarms.
Give me a TL;DR of this text.  
 A 4.2 magnitude earthquake struck the San Francisco area at 4:42am, causing about 2,000 customers to lose power. Although considered a "light" earthquake, it caused some alarm and set off home alarms in Oakland. The shaking lasted about 50 seconds and was felt indoors, but no injuries or damage have been reported.
What's the key takeaway here? 
 The key takeaway is that a magnitude 4.2 earthquake struck the San Francisco area, causing minim

## Text completion

**Exercise**: In this section adjust the `max_completion_tokens` and `temperature` settings below to obtain different responses. Show some examples with the prompt "Continue the story: It was a great time to be alive" with the model "llama-3.1-8b-instant".

* max_completion_tokens - the maximum number of tokens to generate. Note that longer words are made of multiple tokens (set to 200 and 500)
* temperature (positive number) - the higher the number the more random (creative) the output (set to 0.2, 0.8, 2)

In [16]:
# ANSWER (set max_completion_tokens=200, do not have a temperature setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    max_completion_tokens=200,
)

print(response.choices[0].message.content)

It was a great time to be alive, thought Emily as she walked through the bustling streets of New York City. The year was 2025, and technology had surpassed anything she could have imagined. Self-driving cars zoomed by, their sleek designs and bright colors a stark contrast to the crowded sidewalks.

As she walked, Emily's augmented reality glasses beeped, alerting her to a pop-up notification from her friend, Sarah. "Hey, Emily, meet me at Central Park at 3 PM," the message read. "We're going to try out the new virtual reality hiking experience. Can't wait to climb Mount Everest from the comfort of our feet!"

Emily groaned, laughing to herself as she tapped her glasses to acknowledge the message. She had always been an adventurous spirit, and the idea of exploring the world from the comfort of her own neighborhood was both exhilarating and terrifying.

As she continued her walk, Emily passed by a group of people gathered around a holographic display projecting


In [17]:
# ANSWER (set max_completion_tokens=500, do not have a temperature setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    max_completion_tokens=500,
)

print(response.choices[0].message.content)

The sun was shining brightly, casting a warm glow over the bustling streets of the city. The air was alive with the chatter of people from all walks of life, their laughter and conversations weaving a rich tapestry of sound. It was a great time to be alive, indeed.

As I walked down the sidewalk, I couldn't help but feel a sense of excitement and anticipation. The world seemed full of possibility, and I felt like anything was within my grasp. I had just graduated from college, and the possibilities stretched out before me like a map of uncharted territories.

I stopped at a small café to grab a coffee and take in the view. The café was one of those eclectic, bohemian spots that seemed to attract a mix of creatives, entrepreneurs, and free-spirited individuals. The walls were adorned with vibrant artwork and the air was thick with the scent of freshly brewed coffee.

As I sipped my latte, I struck up a conversation with a young woman sitting at a nearby table. She was a freelance writer

In [18]:
# ANSWER (set temperature = 0.2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive."}],
    temperature = 0.2,
)

print(response.choices[0].message.content)

The sun was shining bright, casting a warm glow over the bustling streets of the city. People of all ages and backgrounds walked side by side, each with their own unique story to tell. The air was filled with the sweet scent of blooming flowers and the sound of laughter and music.

As I walked through the crowded streets, I couldn't help but feel a sense of excitement and possibility. It was a great time to be alive, and I was grateful to be a part of it. The world was changing at an incredible pace, and it seemed like anything was possible.

I passed by a group of street performers, who were drawing a large crowd with their acrobatic feats and mesmerizing magic tricks. I watched in awe as they spun plates and juggled balls, their movements fluid and precise.

Next, I walked by a group of artists, who were showcasing their latest works on the sidewalk. Their vibrant paintings and sculptures seemed to come alive in the sunlight, and I couldn't help but be drawn in by their creativity an

In [19]:
# ANSWER (set temperature = 1, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive."}],
    temperature = 1,
)

print(response.choices[0].message.content)

It was a great time to be alive, the sun was shining bright, and the air was filled with the sweet scent of blooming flowers. The year was 1969, and the world was on the cusp of a new era of peace and prosperity. The Apollo 11 mission had just successfully landed on the moon, and the global community was united in their excitement and wonder.

In a small town nestled in the heart of the countryside, a young couple, Emily and Jack, were experiencing the joy and promise of this new era. They had just graduated from college, and were embarking on a journey of self-discovery, love, and adventure.

As they sat on a blanket in the park, watching the children play, Emily turned to Jack with a look of excitement in her eyes. "Can you believe what's happening?" she exclaimed. "The world is changing so fast, and we're right in the middle of it."

Jack smiled, and took her hand in his. "I know, it's an amazing time to be alive. Anything seems possible."

As they talked, they noticed a group of yo

Note what happens when the temperature is set too high!

In [20]:
# ANSWER (set temperature = 2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive."}],
    temperature = 2,
)

print(response.choices[0].message.content)

People of all ages gathered in cities and streets to rejoice in this moment of peace and prosperity. The war seemed like a distant memory, replaced by bustling markets, thriving arts scenes, and a collective sigh of relief.

In one of those markets, Emily stood out among the crowds. Her bright smiles and infectious enthusiasm had made her a local celebrity of sorts, known for her vibrant paintings and infectious laughter. Among her colorful stalls, a large sign caught the eye - 'A Brighter Future for Sale, Price - $100 for a day, or forever, it's up to you'. Passersby laughed at the playful joke until some stopped, curiosity getting the better of them to look more intensely at the artwork displayed alongside it.

Emily had lost her sister some few years ago, during a particularly dark period of war. But now she used those difficult memories as the foundation for something positive: she brought the artwork from an organization called 'Color for Life', where local street art artists and 

### Zero-shot and one-short prompting for question-answering

This section shows the impact of prompting on the response. Zero-shot prompting means we provide the prompt without any examples or additional context. Let us initially ask Mistral a question using no prompting.

In [21]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

When two chemicals react, it's a complex process that involves the interaction of their atoms, molecules, and electrons. Here's a simplified overview:

1. **Chemical Properties**: Each chemical has its own set of chemical properties, such as its reactivity, electronegativity, and molecular structure.
2. **Reaction Conditions**: The reaction conditions, such as temperature, pressure, and concentration, can influence the reaction.
3. **Mechanisms**: Chemical reactions can occur through different mechanisms, such as:
	* **Acid-base reactions**: The transfer of a proton (H+) or an electron pair.
	* **Redox reactions**: The transfer of electrons between two species.
	* **Substitution reactions**: The replacement of an atom or group of atoms.
	* **Condensation reactions**: The combination of two molecules to form a new molecule.
	* **Decomposition reactions**: The breaking down of a molecule into simpler molecules.
4. **Collision Theory**: For a reaction to occur, the reactant molecules must collide with each other at a sufficient energy level, known as the activation energy.
5. **Reaction Rate**: The rate of the reaction depends on the concentration of the reactants, the surface area of the reactants, and the temperature of the reaction.

When two chemicals react, the following steps occur:

1. **Collision**: The reactant molecules collide with each other.
2. **Activation energy**: The colliding molecules must overcome the activation energy barrier to form an intermediate or a transition state.
3. **Intermediate or transition state**: The intermediate or transition state is a temporary arrangement of atoms that is higher in energy than the reactants.
4. **Product formation**: The intermediate or transition state breaks down to form the products.
5. **Product distribution**: The products are distributed according to their equilibrium constants.

Here's a simple example of a chemical reaction between two chemicals, hydrogen gas (H2) and oxygen gas (O2), to form water (H2O):

2H2 (g) + O2 (g) → 2H2O (l)

In this reaction:

1. The hydrogen and oxygen molecules collide with each other.
2. The collided molecules overcome the activation energy barrier to form an intermediate or transition state.
3. The intermediate or transition state breaks down to form water molecules.
4. The water molecules are distributed according to their equilibrium constant.

Keep in mind that chemical reactions are complex processes, and the actual mechanisms can be influenced by various factors, such as catalysts, solvent effects, and reaction conditions.

**Exercise:** Ask the same question but modify the prompt to return the answer to the same question in a simpler form (still using the llama-3.1-8b-instant model). Experiment with different prompts.

In [23]:
# ANSWER

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{
        "role": "user",
        "content": "Explain how two chemicals react, in simple terms for a beginner."
    }],
    temperature=0.7,
)
Markdown(response.choices[0].message.content)

Let's consider a simple example of how two chemicals react: baking soda (sodium bicarbonate) and vinegar (acetic acid).

**Chemical Reaction:**

When you mix baking soda and vinegar, a chemical reaction occurs. Here's what happens:

1. **Baking Soda (Sodium Bicarbonate)**: This is a white, powdery substance that is commonly used in baking.
2. **Vinegar (Acetic Acid)**: This is a clear, liquid substance that is commonly used in cooking and as a cleaning agent.

**The Reaction:**

When you mix baking soda and vinegar, they react to form carbon dioxide gas (a gas that makes things bubbly). This reaction is called a chemical reaction.

Here's a simplified equation:

NaHCO3 (baking soda) + CH3COOH (vinegar) → CO2 (carbon dioxide) + H2O (water)

**What happens:**

When you mix baking soda and vinegar, the following happens:

* The baking soda (sodium bicarbonate) reacts with the vinegar (acetic acid) to form carbon dioxide gas.
* The carbon dioxide gas is released into the air, creating bubbles.
* The mixture also forms water, which is a byproduct of the reaction.

**Result:**

The result of the reaction is a fizzing or bubbling mixture that is often used in science experiments and baking recipes.

**Why it's important:**

This reaction is important because it shows how two chemicals can interact with each other to form new substances. This is a fundamental concept in chemistry, and it's used in many different fields, including cooking, medicine, and engineering.

I hope this helps you understand how two chemicals react! Let me know if you have any other questions.

In [24]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{
        "role": "user",
        "content": "Explain how two chemicals react, in simple terms for a beginner."
    }],
    temperature=0.3,
)
Markdown(response.choices[0].message.content)

Let's consider a simple example of a chemical reaction between two chemicals: baking soda (sodium bicarbonate) and vinegar (acetic acid).

**Chemical Reaction:**

Baking soda (NaHCO3) + Vinegar (CH3COOH) → Carbon dioxide (CO2) + Water (H2O) + Salt (NaAc)

**What happens:**

1. When you mix baking soda and vinegar together, they start to react with each other.
2. The acid in the vinegar (acetic acid) breaks down the baking soda (sodium bicarbonate).
3. This breakdown releases carbon dioxide gas, which is what makes the mixture fizz and bubble.
4. The reaction also produces water and a type of salt called sodium acetate.

**Visual representation:**

Imagine a seesaw with baking soda on one side and vinegar on the other. When you mix them together, the seesaw tips, and the baking soda and vinegar react with each other. The carbon dioxide gas released in the reaction is like a "bubble" that makes the mixture fizz and bubble.

**Key points:**

- Chemical reactions involve the interaction of two or more substances.
- The reactants (baking soda and vinegar) combine to form new substances (carbon dioxide, water, and salt).
- Chemical reactions can release energy in the form of heat, light, or in this case, gas bubbles.

This is a simple example of a chemical reaction, but there are many more complex reactions that occur in the world around us.

In [25]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{
        "role": "user",
        "content": "Explain how two chemicals react, using simple sentences."
    }],
    temperature=0.3,
)
Markdown(response.choices[0].message.content)

Let's consider the reaction between hydrogen peroxide (H2O2) and yeast. 

Yeast is a living organism that contains an enzyme called catalase. 
Catalase breaks down hydrogen peroxide into water (H2O) and oxygen gas (O2).
The reaction is as follows: H2O2 → H2O + O2.

Here's a step-by-step explanation:

1. The yeast cell membrane allows hydrogen peroxide to enter the cell.
2. The catalase enzyme in the yeast cell breaks down the hydrogen peroxide molecule.
3. The hydrogen peroxide molecule is split into two water molecules and one oxygen molecule.
4. The oxygen molecule is released as a gas, creating bubbles.
5. The reaction continues as long as there is hydrogen peroxide present and the yeast cells are alive.

This reaction is an example of a catalytic reaction, where the enzyme (catalase) speeds up the reaction without being consumed by it.

### One-shot prompting ###

Next, note the dramatic change when we give the following template setting a new role and providing an English question followed by a French translation.

In [26]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "system",
             "content": "You translate English to French."},
              {"role": "user",
               "content": "What time is it?"},
               {"role": "assistant",
               "content": "Quelle heure est-il?"},
              {"role": "user",
               "content": "How do two chemicals react?"}],
    temperature = 0.8,
)
print(response.choices[0].message.content)

Comment deux composés chimiques réagissent-ils?


### Few-shot prompting

Recall that since the text generation process outputs one token at a time, their outputs often need adjusting. This is where examples can help.

In [27]:
prompt1 = "I'm gonna head out now, see you later."
response1 = "I will be leaving now. See you later."

prompt2 =  "That movie was super cool!"
response2 = "The movie was very impressive."

prompt3 = "Can't make it to the meeting, sorry."


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a professional editor. Rewrite casual sentences into a formal tone."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


Regrettably, I will be unable to attend the meeting. I apologize for any inconvenience this may cause.


The output can also be moulded to provide SQL output.

In [28]:
prompt1 = "Show me all users who signed up in the last 30 days."
response1 = "SELECT * FROM users WHERE signup_date >= CURRENT_DATE - INTERVAL '30 days';"

prompt2 = "What is the average order value?"
response2 =  "SELECT AVG(order_total) FROM orders;"

prompt3 = "List products that are out of stock."

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are an assistant that translates natural language to SQL."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


SELECT * FROM products WHERE stock_quantity = 0;


**Exercise**: Create a few examples to train the "llama3-70b-8192" LLM to take in user content in the form below and provide output as a pandas dataframe. Use the `exec` function to execute its output to display the answer of sample input as a data frame.

Example:

given the user content

"""

| col1 | col2 | col3

| 32 | 27 | 25

| 64 | 23 | 14

"""

train the model to output

df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})



In [29]:
#ANSWER

#ANSWER

user1 = """col1 | col2 | col3
32 | 27 | 25
64 | 23 | 14
"""

output1 = """
df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})
"""

user2 = """col1 | col2
23 | 12
8 | 76
7 | 5
"""
output2 = """
df = pd.DataFrame({'col1': [23, 8, 7], 'col2': [12, 76, 5]})
"""
user3 = """colA | colB | colC
23 | 12 | 54
8 | 76 | 32
7 | 5 | 3
"""


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a data scientist who will receive data input as a string and provide output as a pandas dataframe called df. Use the examples to guide you"},
        {"role": "user", "content": user1},
        {"role": "assistant", "content": output1},
        {"role": "user", "content": user2},
        {"role": "user", "content": output2},
        {"role": "user", "content": user3}
    ]
)

exec(response.choices[0].message.content.strip()) # string executed as Python code
df

,colA,colB,colC
0,23,12,54
1,8,76,32
2,7,5,3


Also show what happens when the question is asked in the absence of a system role and without few-shot prompting.

In [30]:
# ANSWER
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "user", "content": user3}
    ]
)
response.choices[0].message.content.strip()

"It looks like you're showing a table with three columns: `colA`, `colB`, and `colC`, and three rows of data.\n\nIs there something you'd like to do with this data, such as:\n\n* Perform a calculation or aggregation (e.g. sum, average, min, max)?\n* Filter or sort the data based on certain conditions?\n* Merge this data with another table or dataset?\n* Visualize the data in a chart or graph?\n\nLet me know, and I'll do my best to assist you!"

### Chain-of-thought prompting

The results of question-answering can also be improved by prompting the LLM to provide intermediate steps.

**Exercise**: Using the following prompts, compare the answers of the "llama3-8b-8192" model (set seed=21). (If this model is no longer available choose a model with relatively few parameters.)

zero_shot_prompt = "How many s's are in the word 'success'?"

chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

In [31]:
# ANSWER
# ANSWER
zero_shot_prompt = "How many s's are in the word 'success'?"
chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

response1 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": zero_shot_prompt}],
    seed = 21
)

response2 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": chain_of_thought_prompt}],
    seed = 21
)

print('------zero-shot-prompt------')
print(response1.choices[0].message.content)

print('------chain-of-thought------')
print(response2.choices[0].message.content)

------zero-shot-prompt------
There are 2 s's in the word 'success'.
------chain-of-thought------
To count the number of 's's in the word "success", we'll go through each letter one by one:

1. The word is "succe**s**s".
2. The first letter is 's' (one 's').
3. The second letter is 'u'.
4. The third letter is 'c'.
5. The fourth letter is 'c'.
6. The fifth letter is 'e'.
7. The sixth letter is 's' (second 's').
8. The seventh letter is 's' (third 's').

After counting the letters, we have three 's's in the word "success".


## Comparison of LLMs

**Exercise**: Compare the performance of 2 LLMs by outputting the answers of the following questions into a dataframe.

    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",

Column headings:

Model Name | Question | Answer

In [32]:
pd.set_option('display.max_colwidth', None) # allows wide dataframes to be viewed
models = ["gemma2-9b-it", "llama-3.1-8b-instant"] #can edit this

# ANSWER
prompts = [
    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",
]

results = {'Model Name': [], 'Question': [], 'Answer': []}

for model in models:
    for prompt in prompts:
        results['Model Name'].append(model)
        results['Question'].append(prompt)
        try:
            output = client.chat.completions.create(model = model, messages=[{"role": "user", "content": prompt}])
            results['Answer'].append(output.choices[0].message.content.strip())

        except Exception as e:
            print(f"Error with {model}: {e}")
            results['Answer'].append((prompt, "ERROR"))


df = pd.DataFrame(results)
df

,Model Name,Question,Answer
0,gemma2-9b-it,Tell me a joke about data science.,"Why did the data scientist break up with the statistician? \n\nBecause they had too many ""significant differences""! 😂"
1,gemma2-9b-it,How can one calculate 22 * 13 mentally?,Here's a way to calculate 22 * 13 mentally using a bit of cleverness:\n\n**1. Break down the numbers:**\n\n* Think of 22 as (20 + 2) \n\n**2. Distribute:**\n\n* (20 + 2) * 13 \n* = 20 * 13 + 2 * 13\n\n**3. Calculate the easier parts:**\n\n* 20 * 13 = 260\n* 2 * 13 = 26\n\n**4. Add the results:**\n\n* 260 + 26 = 286\n\n\n\nLet me know if you'd like to try another mental math problem!
2,gemma2-9b-it,Write a creative story about a baby learning to crawl.,"Bartholomew ""Bartleby"" Buttercup was a momentous baby. Not just in the way that all babies are momentous, with their first coos and gummy grins, but in his sheer determination to learn to crawl. His older sister, Poppy, zoomed around the room like a furry, four-legged rocket, leaving trails of building blocks and dropped crayons in her wake. Bartleby, however, remained rooted, surrounded by a galaxy of toys he desperately wanted to reach. \n\nHis movements were epic, if a little slow. He’d fling his fists at the air, his chubby legs kicking out with the force of a startled frog. He'd squirm and wiggle, sending his bright red diaper into a frenzied dance. But those chubby, determined legs refused to cooperate in the synchronized effort required for crawling.\n\nOne sunny afternoon, Poppy, in her usual whirlwind fashion, toppled over Bartleby’s beloved stuffed caterpillar, Mr. Flutterkins. Mr. Flutterkins landed on his back, his tiny felt antennae dangling pathetically. A pained gurgle escaped Bartleby’s lips. Poppy, ever the adventurer, scrambled over the fallen toy, oblivious to the distress she'd caused. \n\nBartleby watched her with a mixture of awe and frustration. His eyes, wide and blue like a summer sky, drilled into the back of Poppy's head. Then, a strange thing happened. His legs, in a sudden burst of unexpected action, propelled him forward. \n\nHe bumped, he slid, he wobbled, but he was moving. He was crawling! \n\nPoppy, mid-adventure, paused, her head swiveling to look at Bartleby. Her eyes widened in surprise. Bartleby, fueled by a primal urge to rescue Mr. Flutterkins, kept going. He bumped into a stack of books, sending them cascading to the floor. He wriggled under the coffee table, narrowly avoiding a collision with a stray sock.\n\nFinally, he reached Mr. Flutterkins, his grubby hands scooping up the fallen toy. He held Mr. Flutterkins aloft, his face beaming with triumph. \n\nPoppy, forgetting her own adventures, clambered down and joined him, giggling. They played together, Bartleby pulling himself forward in a jerky, determined crawl, while Poppy chased him, her laughter echoing through the room.\n\n\nBartleby, the once stationary baby, had conquered the holy grail of crawling. He’d even done it in his own unique way, fuelled by love for a stuffed caterpillar and a fierce resolve to move. And somewhere in that joyous chaos, under the watchful gaze of a happy mother, another monumental moment blossomed - the moment siblings truly understood each other."
3,llama-3.1-8b-instant,Tell me a joke about data science.,Why did the data scientist quit his job?\n\nBecause he didn't get any data out of his decision.
4,llama-3.1-8b-instant,How can one calculate 22 * 13 mentally?,"To calculate 22 * 13 mentally, you can use a technique called the ""nines trick"" in combination with estimation. However, a simpler method involves breaking down the multiplication into known numbers and using some mental math techniques.\n\nMethod 1: Break down the numbers\n\nFirst, break down 13 into a sum of 10 and 3:\n22 * (10 + 3) = 22 * 10 + 22 * 3\n\nNext, calculate the individual parts:\n- 22 * 10 = 220 (multiply 22 by 20 and add the remaining 2)\n 22 * 20 = 440\n Add the remaining 2: 440 + 2 = 442\n- Since the actual multiplication w

### Bonus

See if you can prompt an LLM to perform sentiment analysis (output 'Positive' or 'Negative' only) on a given piece of text.

In [33]:
# ANSWER
input1 = "I was inspired by the actor's performance."
output1 = "Positive"

input2 = "The traffic is so heavy today."
output2 = "Negative"

input3 = "He opened a business today."


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are amazing at sentiment analysis. Give the sentiment of the next sentence as the examples show."},
        {"role": "user", "content": input1},
        {"role": "assistant", "content": output1},
        {"role": "user", "content": input2},
        {"role": "assistant", "content": output2},
        {"role": "user", "content": input3},
    ]
)
response.choices[0].message.content

'Positive'

## Conclusion

We worked with a few Large Language Models (LLMs) using Groq and experimented with prompting for summarisation, text completion and question-answering tasks.

We also explored controlling the randomness (creativity) of output through the temperature setting and tried different types of prompting to achieve desired forms of output.

## References
1. [Groq's prompting guide](https://console.groq.com/docs/prompting)
2. [Groq's playground](https://console.groq.com/playground)



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



